In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
import scipy.optimize as opt
from selenium.webdriver.common.by import By
import numpy as np
import astropy.units as u
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
path = '/Users/wendy/_Lib/chromedriver_mac_arm64/chromedriver'

In [3]:
driver = webdriver.Chrome(path)


/var/folders/1c/5vhp7f916wv452z62sq58lzc0000gn/T/ipykernel_85861/2459958106.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [4]:
driver.get('https://smm.misis.ru/CPW-resonator-coupling/')


In [5]:
freqs = [6310.460308,6599.399983,6753.384976,6927.24498,7175.912057,7462.971308,
         7541.496634,7782.320107,7987.50912,8171.432504]
freqs = np.array(freqs)

In [6]:
def set_epsilon(epsilon):
    epsi = driver.find_element(By.ID, 'epsilon')
    epsi.clear()
    epsi.send_keys('11.45')

In [7]:
def single_cpw_impedance(pin, gap):
    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    AB = input_box.find_element(By.NAME, 'AB')
    BC = input_box.find_element(By.NAME, 'BC')
    CD = input_box.find_element(By.NAME, 'CD')

    AB.clear()
    AB.send_keys(gap)

    CD.clear()
    CD.send_keys(gap)

    BC.clear()
    BC.send_keys(pin)


    impedance_box = driver.find_element(By.ID, 'impedance-data')
    impedance = impedance_box.get_attribute('innerHTML')



    string = impedance
    match = re.search(r'Z0:(\d+\.\d+)', string)

    if match:
        number = float(match.group(1))
        # print(number)
    else:
        print("No match found.")


    return number

    

In [8]:
set_epsilon(11.45)

In [9]:
single_cpw_impedance(12.4, 6.99)

49.99140914897713

In [10]:
def find_optimal_gap(pin):
    def f(gap):
        time.sleep(0.01)
        return single_cpw_impedance(pin, gap)-50
    gap_val = opt.brentq(f, 0, 10)
    return round(gap_val, 2)

In [11]:
resonator_pin = 12.4
transmission_line_pin = 12.4
resonator_gap = find_optimal_gap(resonator_pin)
transmission_line_gap = find_optimal_gap(transmission_line_pin)

After getting optimal values for the individual transmissionline/resonators, we optimize the coupling between them.

In [12]:
def set_up_structure(coupling_space,resonator_pin = resonator_pin,resonator_gap=resonator_gap,transmission_line_pin=transmission_line_pin, transmission_line_gap=transmission_line_gap):
    button = driver.find_element(By.ID, 'add-conductor')
    button.click()
    button.click()
    time.sleep(0.5)
    conductor1 = driver.find_element(By.ID, 'tc3')
    conductor1.click()
    conductor1.click()
    time.sleep(0.5)

    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    AB = input_box.find_element(By.NAME, 'AB')
    BC = input_box.find_element(By.NAME, 'BC')
    CD = input_box.find_element(By.NAME, 'CD')
    DE = input_box.find_element(By.NAME, 'DE')
    EF = input_box.find_element(By.NAME, 'EF')
    FG = input_box.find_element(By.NAME, 'FG')
    GH = input_box.find_element(By.NAME, 'GH')

    AB.clear()
    AB.send_keys(transmission_line_gap)

    CD.clear()
    CD.send_keys(transmission_line_gap)

    BC.clear()
    BC.send_keys(transmission_line_pin)
    time.sleep(0.5)

    
    DE.clear()
    DE.send_keys(coupling_space)

    EF.clear()
    EF.send_keys(resonator_gap)

    FG.clear()
    FG.send_keys(resonator_pin)

    GH.clear()
    GH.send_keys(resonator_gap)
    

In [13]:
def change_gap(coupling_space):
    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    DE = input_box.find_element(By.NAME, 'DE')
    DE.clear()
    DE.send_keys(coupling_space)

In [14]:
def set_up_resonator():
    ztl1 = driver.find_element(By.ID, 'Zt1_short')
    ztl1.click()
    ztl2 = driver.find_element(By.ID, 'Zt2_open')
    ztl2.click()

    SC_coupler_len = driver.find_element(By.ID, 'lc')
    SC_coupler_len.clear()
    SC_coupler_len.send_keys('0.17')

    SC_coupler_l1 = driver.find_element(By.ID, 'l1')
    SC_coupler_l1.clear()
    SC_coupler_l1.send_keys('0')

    SC_coupler_l2 = driver.find_element(By.ID, 'l2')
    SC_coupler_l2.clear()
    SC_coupler_l2.send_keys('4')

In [15]:
set_up_structure(5)

In [16]:
set_up_resonator()

In [17]:
def find_resonator_length(freq):
    def f(length):
        time.sleep(0.01)
        SC_coupler_len = driver.find_element(By.ID, 'lc')
        SC_coupler_len.clear()
        SC_coupler_len.send_keys(length)
        return single_cpw_impedance(12.4, 9)-freq
    length_val = opt.brentq(f, 0, 10, xtol=0.001)
    return round(length_val, 2)

In [18]:
def find_resonator_specs():

    table = driver.find_element(By.ID, 'resonances')
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')

    # Extracting column names from HTML table headers
    cols = []
    for th in soup.find_all('th'):
        cols.append(th.text.strip())
        
    # Extracting table rows from HTML table body
    data = []
    for tr in soup.find_all('tr')[1:]:
        row = []
        for td in tr.find_all('td'):
            row.append(td.text.strip())
        data.append(row)

    # Creating pandas dataframe from table rows and columns
    df = pd.DataFrame(data, columns=cols)

    return df
    
    

In [19]:
freq = float(find_resonator_specs()['Frequency, GHz'][0])

In [20]:
def find_freq(len):
    SC_coupler_l2 = driver.find_element(By.ID, 'l2')
    SC_coupler_l2.clear()
    SC_coupler_l2.send_keys(len)

    return float(find_resonator_specs()['Frequency, GHz'][0])

In [21]:
def find_resonator_length(freq):
    def f(length):
        time.sleep(0.01)
        SC_coupler_len = driver.find_element(By.ID, 'l2')
        SC_coupler_len.clear()
        SC_coupler_len.send_keys(length)
        return float(find_resonator_specs()['Frequency, GHz'][0])-freq
    length_val = opt.brentq(f, 0, 10, xtol=0.001)
    return round(length_val, 2)

    

In [22]:
freqs[0]

6310.460308

In [23]:
l = find_resonator_length(freqs[0]/1000)

In [24]:
freq_len = {}
for freq in freqs/1000:
    freq_len[freq] = find_resonator_length(freq)

In [25]:
freq_len

{6.310460308: 4.59,
 6.5993999830000005: 4.38,
 6.753384976: 4.28,
 6.92724498: 4.17,
 7.175912057: 4.02,
 7.462971308: 3.85,
 7.5414966340000005: 3.81,
 7.782320106999999: 3.69,
 7.9875091199999995: 3.59,
 8.171432504: 3.51}

In [26]:
def find_Q(gap,freq):

    change_gap(gap)
    
    length = find_resonator_length(freq)
    SC_coupler_len = driver.find_element(By.ID, 'l2')
    SC_coupler_len.clear()
    SC_coupler_len.send_keys(length)

    
    return float(find_resonator_specs()['Quality factor'][0])

    


In [27]:
freqs[0]

6310.460308

In [28]:
Q = find_Q(1,freqs[0]/1000)

In [29]:
kappa = 2*np.pi*freq*u.GHz/Q

In [30]:
kappa.to(u.MHz)

<Quantity 2.69883435 MHz>

In [31]:
def find_optimal_gap(freq):
    def kappa(gap):
        time.sleep(0.01)
        return (2*np.pi*freq*u.GHz/find_Q(gap,freq)-1*u.MHz).to(u.MHz).value
    gap_val = opt.brentq(kappa, 0, 15, xtol=0.001)
    return round(gap_val, 2)

In [32]:
find_optimal_gap(freqs[0]/1000)

3.9

In [34]:
gaps = {}
gapps = []

for freq in freqs/1000:
    gaps[freq] = find_optimal_gap(freq)
    gapps.append(gaps[freq])

In [35]:
gaps

{6.310460308: 3.9,
 6.5993999830000005: 4.58,
 6.753384976: 4.94,
 6.92724498: 5.36,
 7.175912057: 5.96,
 7.462971308: 6.68,
 7.5414966340000005: 6.87,
 7.782320106999999: 7.45,
 7.9875091199999995: 7.96,
 8.171432504: 8.4}

In [36]:
gapps

[3.9, 4.58, 4.94, 5.36, 5.96, 6.68, 6.87, 7.45, 7.96, 8.4]

In [37]:
dat = pd.DataFrame(gaps, index = [0])
dat.to_csv('gaps_um.csv')

In [38]:
dat.values[0]

array([3.9 , 4.58, 4.94, 5.36, 5.96, 6.68, 6.87, 7.45, 7.96, 8.4 ])

In [66]:
data_all = pd.read_csv(r'/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_test2').drop(columns = ['Unnamed: 0'])

In [67]:
data_all

,Target_qb_Freq(GHz),Ljs,Ljs (nH),Offsets (mm),TQx,TQy,Small,TQ_mir,CPW_freq_predict(GHz),Coupling_gap(um),Coupling_gap_feedline(um),Sizes (um),Buffers (um),Coupling_len(um),Coupling_len
0,4.0,13nH,13,0.000,-0.267mm,0.45mm,False,True,6.401282,15,1.11,224.327401,237.672599,69.878079,69.87807929705554um
1,4.2,13nH,13,0.000,-0.2535mm,0.44mm,False,True,6.689526,15,1.54,203.923344,231.076656,58.245181,58.24518148422307um
2,4.4,10nH,10,0.000,-0.2485mm,0.43mm,False,True,6.796263,15,1.78,235.170778,189.829222,58.658030,58.6580296993377um
3,4.6,10nH,10,0.050,0.3675mm,0.5mm,True,False,6.993693,15,2.06,213.630207,281.369793,67.261193,67.2611926131531um
4,4.8,8nH,8,0.055,0.3605mm,0.458mm,True,False,7.212317,15,2.48,238.493760,242.506240,73.386058,73.38605790089028um
5,5.0,8nH,8,0.055,0.3485mm,0.45mm,True,False,7.495441,15,3.01,218.851425,238.148575,62.802910,62.80290956338173um
6,5.2,8nH,8,0.042,0.3425mm,0.46mm,True,False,7.593559,15,3.14,200.242799,244.757201,51.557798,51.5577977048421um
7,5.4,6nH,6,0.065,0.3375mm,0.46mm,True,False,7.799704,15,3.57,236.923095,198.076905,56.360661,56.3606611629999um
8,5.6,6nH,6,0.055,0.329mm,0.44mm,True,False,8.006815,20,3.95,217.997190,200.002810,69.714398,69.71439788499663um
9,5.8,6nH,6,0.200,0.343mm,0.45mm,True,False,8.204176,20,4.28,200.537671,245.462329,71.499159,71.49915894415778um


In [68]:
data_all['Coupling_gap_feedline(um)'] = gapps

In [114]:
data_all['TQx'] = modified


In [115]:
data_all.to_csv(r'/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_test3.csv')

In [84]:
original = data_all['TQx'].values
small = data_all['Small'].values

In [85]:
small

array([False, False, False,  True,  True,  True,  True,  True,  True,
        True])

In [111]:
modified = []
for i, x in enumerate(original):
    dis = 0.0125
    if small[i]:
        dis = -0.0625+0.025
    v = np.round(float(x[:-2]) + dis, decimals = 5)
    modified.append(str(v)+' mm')

In [112]:
original

array(['-0.2545 mm', '-0.241 mm', '-0.236 mm', '0.38 mm', '0.373 mm',
       '0.361 mm', '0.355 mm', '0.35 mm', '0.3415 mm', '0.3555 mm'],
      dtype=object)

In [113]:
modified

['-0.242 mm',
 '-0.2285 mm',
 '-0.2235 mm',
 '0.3425 mm',
 '0.3355 mm',
 '0.3235 mm',
 '0.3175 mm',
 '0.3125 mm',
 '0.304 mm',
 '0.318 mm']

In [ ]:
conductor1 = driver.find_element(By.ID, 'tc3')

In [ ]:
conductor1.click()

In [ ]:
conductor1.click()

In [ ]:
input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
AB = input_box.find_element(By.NAME, 'AB')
BC = input_box.find_element(By.NAME, 'BC')
CD = input_box.find_element(By.NAME, 'CD')
DE = input_box.find_element(By.NAME, 'DE')
EF = input_box.find_element(By.NAME, 'EF')
FG = input_box.find_element(By.NAME, 'FG')
GH = input_box.find_element(By.NAME, 'GH')

In [ ]:
AB.clear()
AB.send_keys('9')
CD.clear()
CD.send_keys('9')

BC.clear()
BC.send_keys('15.95')

DE.clear()
DE.send_keys('5')

EF.clear()
EF.send_keys('7.4')

FG.clear()
FG.send_keys('12.5')

GH.clear()
GH.send_keys('7.4')



In [ ]:
epsi = driver.find_element(By.ID, 'epsilon')
epsi.clear()
epsi.send_keys('11.45')

In [ ]:
SC_coupler_len = driver.find_element(By.ID, 'lc')
SC_coupler_len.clear()
SC_coupler_len.send_keys('0.12')

SC_coupler_l1 = driver.find_element(By.ID, 'l1')
SC_coupler_l1.clear()
SC_coupler_l1.send_keys('0')

SC_coupler_l2 = driver.find_element(By.ID, 'l2')
SC_coupler_l2.clear()
SC_coupler_l2.send_keys('4')



In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# Extracting column names from HTML table headers
cols = []
for th in soup.find_all('th'):
    cols.append(th.text.strip())
    
# Extracting table rows from HTML table body
data = []
for tr in soup.find_all('tr')[1:]:
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    data.append(row)

# Creating pandas dataframe from table rows and columns
df = pd.DataFrame(data, columns=cols)

print(df)